# 데이터 불러오기

## 모듈 불러오기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import plotly.express as px
import plotly.graph_objects as go

## 파일 불러오기

In [ ]:
# isc_2_1='isc_2_1'
# isc_3_1='isc_3_1'
# isc_3_3='isc_3_3'
nsc_1_1='nsc_1_1'
# nsc_1_2='nsc_1_2'
# nsc_2_5='nsc_2_5'
# sdc_1_1='sdc_1_1'
# sdc_2_1='sdc_2_1'
# sic_2_6='sic_2_6'
# sic_4_2='sic_4_2'

choosefile = nsc_1_1
address = 'https://raw.githubusercontent.com/hanyeob/fine_dust/master/csv%20files/' + choosefile +".csv"

In [ ]:
dataset = pd.read_csv(address)

In [ ]:
dataset[0:5]

## 데이터 정리

In [ ]:
dataset = dataset.loc[:,'tt':'CO2_classroom']
dataset = dataset.drop(columns=['temp_outdoor', 'RH_outdoor','AH_outdoor','temp_classroom', 'RH_classroom','AH_classroom'])
dataset

In [ ]:
dataset['tt'] = pd.to_datetime(dataset['tt'], format="%d-%m-%Y %H:%M:%S")
dataset.tt

## 불러온 데이터 NaN값 처리

## NaN값 확인

In [ ]:
dataset.isnull().sum()

### NaN값을 이후/이전에 측정된 값으로 채워넣기 

In [ ]:
for i in reversed(range(0,len(dataset))): 
    if(np.isnan(dataset.loc[i,'CO2_classroom'])):
      if(i+1 < len(dataset)):
        dataset.loc[i,'CO2_classroom'] = dataset.loc[i+1,'CO2_classroom']

for i in range(0,len(dataset)): 
    if(np.isnan(dataset.loc[i,'CO2_classroom'])):
      if(i > 0):
        dataset.loc[i,'CO2_classroom'] = dataset.loc[i-1,'CO2_classroom']

# 미세먼지, 초미세먼지 단위 맞춤 (Scaling)

In [ ]:
dataset['0.3_outdoor_scaled'] = dataset['0.3_outdoor'] * 0.3 ** 3
dataset['0.5_outdoor_scaled'] = dataset['0.5_outdoor'] * 0.5 ** 3
dataset['1.0_outdoor_scaled'] = dataset['1.0_outdoor'] * 1.0 ** 3
dataset['3.0_outdoor_scaled'] = dataset['3.0_outdoor'] * 3.0 ** 3
dataset['5.0_outdoor_scaled'] = dataset['5.0_outdoor'] * 5.0 ** 3
dataset['10.0_outdoor_scaled'] = dataset['10.0_outdoor'] * 10.0 ** 3

dataset['0.3_classroom_scaled'] = dataset['0.3_classroom'] * 0.3 ** 3
dataset['0.5_classroom_scaled'] = dataset['0.5_classroom'] * 0.5 ** 3
dataset['1.0_classroom_scaled'] = dataset['1.0_classroom'] * 1.0 ** 3
dataset['3.0_classroom_scaled'] = dataset['3.0_classroom'] * 3.0 ** 3
dataset['5.0_classroom_scaled'] = dataset['5.0_classroom'] * 5.0 ** 3
dataset['10.0_classroom_scaled'] = dataset['10.0_classroom'] * 10.0 ** 3

dataset

## 미세먼지/초미세먼지(내부,외부) 데이터 통합

In [ ]:
dataset['PM2.5_outdoor'] = np.sum(dataset.loc[:,'0.3_outdoor_scaled':'1.0_outdoor_scaled'], axis=1)
dataset['PM10_outdoor'] =  np.sum(dataset.loc[:,'3.0_outdoor_scaled':'10.0_outdoor_scaled'], axis=1)

dataset['PM2.5_classroom'] = np.sum( dataset.loc[:,'0.3_classroom_scaled':'1.0_classroom_scaled'], axis=1)
dataset['PM10_classroom'] = np.sum( dataset.loc[:,'3.0_classroom_scaled':'10.0_classroom_scaled'], axis=1)

dataset

In [ ]:
# CO2_fig = go.Figure()
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_outdoor'],mode='lines',name='CO2_outdoor'))
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_classroom'],mode='lines',name='CO2_classroom'))

In [ ]:
# PM_fig = go.Figure()
# from plotly.subplots import make_subplots
# PM_fig = make_subplots(specs=[[{"secondary_y": True}]])

# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM2.5_outdoor'],line=dict(color='red'), mode='lines',name='PM2.5_outdoors'))
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM10_outdoor'],line=dict(color='blue'),mode='lines',name='PM10_outdoors'),secondary_y=True,)
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM2.5_classroom'],line=dict(color='orange'),mode='lines',name='PM2.5_classroom'))
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM10_classroom'],line=dict(color='skyblue'),mode='lines',name='PM10_classroom'),secondary_y=True,)

# PM_fig.update_yaxes(title_text="PM 2.5", secondary_y=False)
# PM_fig.update_yaxes(title_text="PM10", secondary_y=True)

# 교실 내 변화량 column생성

In [ ]:
for i in range(1,len(dataset)) :
    dataset.loc[i,'PM2.5_classroom_change'] = dataset.loc[i,'PM2.5_classroom'] - dataset.loc[i-1,'PM2.5_classroom']
    dataset.loc[i,'PM10_classroom_change'] = dataset.loc[i,'PM10_classroom'] - dataset.loc[i-1,'PM10_classroom']
    dataset.loc[i,'CO2_classroom_change'] = dataset.loc[i,'CO2_classroom'] - dataset.loc[i-1,'CO2_classroom']

dataset.loc[0,'PM2.5_classroom_change'] =dataset.loc[1,'PM2.5_classroom_change']
dataset.loc[0,'PM10_classroom_change']  =dataset.loc[1,'PM10_classroom_change'] 
dataset.loc[0,'CO2_classroom_change']   =dataset.loc[1,'CO2_classroom_change']  

In [ ]:
dataset.loc[:,['CO2_classroom','CO2_classroom_change']]

In [ ]:
# CO2_fig = go.Figure()
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_outdoor'],mode='lines',name='CO2_outdoor'))
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_classroom'],mode='lines',name='CO2_classroom'))

# 스케일링 (일반화)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaled_data = scaler.fit_transform(dataset[
                ['CO2_outdoor','PM2.5_outdoor','PM10_outdoor',
               'CO2_classroom','PM2.5_classroom','PM10_classroom',
               'CO2_classroom_change','PM2.5_classroom_change','PM10_classroom_change']
              ])
scaled_data = pd.DataFrame(data=scaled_data,columns=['CO2_outdoor','PM2.5_outdoor','PM10_outdoor',
               'CO2_classroom','PM2.5_classroom','PM10_classroom',
               'CO2_classroom_change','PM2.5_classroom_change','PM10_classroom_change'])
scaled_data.insert(loc=0,column='tt',value=dataset['tt'])
scaled_data

In [ ]:
scaled_data['CO2_classroom_change'].median()

In [ ]:
px.histogram(x =scaled_data['CO2_classroom_change'])

# 라벨링


## 재실여부 라벨링

In [ ]:
scaled_data['attend_manual'] = np.NaN

In [ ]:
for i in range(1,len(scaled_data)) :
        # 교실내 CO2가 외부 CO2보다 많으면서 교실내 CO2양 증가 
    if((scaled_data.loc[i,'CO2_classroom_change'] > scaled_data['CO2_classroom_change'].median()) & (scaled_data.loc[i,'CO2_classroom'] > scaled_data.loc[i,'CO2_outdoor'])) :
        scaled_data.loc[i,'attend_manual'] = 'Yes';
        # 교실내 PM10이 외부 PM10보다 많으면서 교실내 PM10양 증가 
    elif((scaled_data.loc[i,'PM10_classroom_change'] > scaled_data['PM10_classroom_change'].median()) & (scaled_data.loc[i,'PM10_classroom'] > scaled_data.loc[i,'PM10_outdoor'])):
        scaled_data.loc[i,'attend_manual'] = 'Yes';
    

In [ ]:
# 하교시간
scaled_data.loc[((np.datetime64('2019-10-28T18:59') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-29T08:20'))),'attend_manual'] = 'No'
scaled_data.loc[((np.datetime64('2019-10-29T16:40') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-30T07:50'))),'attend_manual'] = 'No'
scaled_data.loc[((np.datetime64('2019-10-30T16:50') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-31T08:20'))),'attend_manual'] = 'No'
scaled_data.loc[((np.datetime64('2019-10-31T16:45') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-01T08:22'))),'attend_manual'] = 'No'
scaled_data.loc[((np.datetime64('2019-11-01T15:36') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-02T11:10'))),'attend_manual'] = 'No'
scaled_data.loc[((np.datetime64('2019-11-02T13:00') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-04T08:03'))),'attend_manual'] = 'No'
scaled_data.loc[((np.datetime64('2019-11-04T16:59') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-05T07:30'))),'attend_manual'] = 'No'

# 외부수업
scaled_data.loc[((np.datetime64('2019-10-31T09:45') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-31T10:27'))),'attend_manual'] = 'No'



In [ ]:
px.line(x=scaled_data.tt, y= scaled_data.attend_manual)

## 환기여부 라벨링

In [ ]:
scaled_data['vent_manual'] = np.NaN

In [ ]:
# CO2_fig = go.Figure()
# CO2_fig.add_trace(go.Scatter(x=scaled_data['tt'], y=scaled_data['CO2_outdoor'],mode='lines',name='CO2_outdoor'))
# CO2_fig.add_trace(go.Scatter(x=scaled_data['tt'], y=scaled_data['CO2_classroom'],mode='lines',name='CO2_classroom'))

In [ ]:
for i in range(1,len(scaled_data)) :
        # 교실내 PM2,5가 외부 PM2.5보다 적었는데 교실내 PM2.5양 증가 
    if((scaled_data.loc[i,'PM2.5_classroom_change'] > scaled_data['PM2.5_classroom_change'].median()) & (scaled_data.loc[i,'PM2.5_classroom'] < scaled_data.loc[i,'PM2.5_outdoor'])) :
        scaled_data.loc[i,'vent_manual'] = 'Yes';
    

In [ ]:
# 하교시간
scaled_data.loc[((np.datetime64('2019-10-28T18:59') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-29T08:20'))),'vent_manual'] = 'No'
scaled_data.loc[((np.datetime64('2019-10-29T16:40') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-30T07:50'))),'vent_manual'] = 'No'
scaled_data.loc[((np.datetime64('2019-10-30T16:50') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-31T08:20'))),'vent_manual'] = 'No'
scaled_data.loc[((np.datetime64('2019-10-31T16:45') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-01T08:22'))),'vent_manual'] = 'No'
scaled_data.loc[((np.datetime64('2019-11-01T15:36') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-02T11:10'))),'vent_manual'] = 'No'
scaled_data.loc[((np.datetime64('2019-11-02T13:00') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-04T08:03'))),'vent_manual'] = 'No'
scaled_data.loc[((np.datetime64('2019-11-04T16:59') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-05T07:30'))),'vent_manual'] = 'No'

# CO2 그래프 판단
scaled_data.loc[((np.datetime64('2019-10-29T09:46') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-29T09:49'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-10-29T10:11') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-29T10:14'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-10-29T10:35') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-29T10:53'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-10-29T12:11') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-29T12:39'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-10-29T12:51') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-29T13:04'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-10-29T14:10') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-29T14:21'))),'vent_manual'] = 'Yes'

scaled_data.loc[((np.datetime64('2019-10-30T09:39') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-30T09:47'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-10-30T10:16') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-30T11:09'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-10-30T11:48') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-30T12:22'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-10-30T12:44') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-30T15:00'))),'vent_manual'] = 'Yes'

scaled_data.loc[((np.datetime64('2019-10-31T08:53') < scaled_data['tt'])         
             &(scaled_data['tt'] < np.datetime64('2019-10-31T10:27'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-10-31T11:32') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-31T12:58'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-10-31T13:45') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-10-31T14:12'))),'vent_manual'] = 'Yes'

scaled_data.loc[((np.datetime64('2019-11-01T12:36') < scaled_data['tt'])              
             &(scaled_data['tt'] < np.datetime64('2019-11-01T12:50'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-11-01T13:45') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-01T14:28'))),'vent_manual'] = 'Yes'

scaled_data.loc[((np.datetime64('2019-11-02T11:55') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-02T12:14'))),'vent_manual'] = 'Yes'

scaled_data.loc[((np.datetime64('2019-11-04T09:41') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-04T10:26'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-11-04T11:22') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-04T11:48'))),'vent_manual'] = 'Yes'
scaled_data.loc[((np.datetime64('2019-11-04T12:08') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-04T16:28'))),'vent_manual'] = 'Yes'

scaled_data.loc[((np.datetime64('2019-11-05T09:42') < scaled_data['tt']) 
             &(scaled_data['tt'] < np.datetime64('2019-11-05T10:30'))),'vent_manual'] = 'Yes'



In [ ]:
sum(scaled_data['vent_manual'].isnull())

In [ ]:
px.line( y = scaled_data.vent_manual)

## NaN값 처리

In [ ]:
# null값 설정 : 재실중, 환기X
scaled_data['attend_manual'] = scaled_data['attend_manual'].fillna('Yes')
scaled_data['vent_manual'] = scaled_data['vent_manual'].fillna('No')

#null값 삭제
# scaled_data = scaled_data.dropna(how='any',axis=0) 

In [ ]:
scaled_data

## CSV저장

In [ ]:
ML_source = pd.get_dummies(scaled_data); 
# scaled_data_data.to_csv(nsc_1_1+'_labeled_scaled_data.csv',index=False)
ML_source

In [ ]:
ML_source.sum()

#머신러닝 모델 생성

## 모듈 불러오기

In [ ]:
import tensorflow as tf

## 독립변수, 종속변수 분리

In [ ]:
독립 = ML_source[['CO2_outdoor', 'PM2.5_outdoor', 'PM10_outdoor', 'CO2_classroom',
       'PM2.5_classroom', 'PM10_classroom', 'CO2_classroom_change',
       'PM2.5_classroom_change', 'PM10_classroom_change']]
재실여부 = ML_source[['attend_manual_No', 'attend_manual_Yes']]
환기여부 = ML_source[['vent_manual_No', 'vent_manual_Yes']]
print(독립.shape,재실여부.shape, 환기여부.shape)

## 레이어 생성

In [ ]:
attendX = tf.keras.layers.Input(shape=[9])

attendH = tf.keras.layers.Dense(7)(attendX)
attendH = tf.keras.layers.BatchNormalization()(attendH)
attendH = tf.keras.layers.Activation('relu')(attendH)

attendY = tf.keras.layers.Dense(2, activation='softmax')(attendH)

attend_model = tf.keras.models.Model(attendX, attendY)
attend_model.compile(loss='categorical_crossentropy',metrics='accuracy')

In [ ]:
ventX = tf.keras.layers.Input(shape=[9])

ventH = tf.keras.layers.Dense(7)(ventX)
ventH = tf.keras.layers.BatchNormalization()(ventH)
ventH = tf.keras.layers.Activation('relu')(ventH)

ventY = tf.keras.layers.Dense(2, activation='softmax')(ventH)

vent_model = tf.keras.models.Model(ventX, ventY)
vent_model.compile(loss='categorical_crossentropy',metrics='accuracy')

## 모델 피팅

In [ ]:
attend_model.fit(독립, 재실여부, epochs=100 ) #verbose = 1

In [ ]:
vent_model.fit(독립, 환기여부, epochs=100 ) #verbose = 1

## 모델 저장


In [ ]:
attend_model.save('nsc_1-1_based_attendModel.h5')
vent_model.save('nsc_1-1_based_ventModel.h5')